## 1. 데이터 준비와 전처리

In [6]:
import pandas as pd
PATH = '/content/drive/MyDrive/Github/AIFFEL/E09'

In [7]:
rating_file_path= PATH + '/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [9]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [10]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [11]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path= PATH + '/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


2) 분석해 봅시다.
ratings에 있는 유니크한 영화 개수
rating에 있는 유니크한 사용자 수
가장 인기 있는 영화 30개(인기순)



3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.



4) CSR matrix를 직접 만들어 봅시다.



5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.



6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.



7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.



8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.